<a href="https://colab.research.google.com/github/jkm2000korea/master_degree/blob/main/13%EC%A3%BC%EC%B0%A8_Association_Analysis_20240527.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/데이터마이닝_2024/예제 데이터')
import pandas as pd
import numpy as np

Mounted at /content/drive


In [ ]:
df1 = pd.read_csv('Faceplate.csv')
df1

,Transaction,Red,White,Blue,Orange,Green,Yellow
0,1,1,1,0,0,1,0
1,2,0,1,0,1,0,0
2,3,0,1,1,0,0,0
3,4,1,1,0,1,0,0
4,5,1,0,1,0,0,0
5,6,0,1,1,0,0,0
6,7,1,0,1,0,0,0
7,8,1,1,1,0,1,0
8,9,1,1,1,0,0,0
9,10,0,0,0,0,0,1


In [ ]:
df1.set_index('Transaction', inplace=True)
print(df1)

             Red  White  Blue  Orange  Green  Yellow
Transaction                                         
1              1      1     0       0      1       0
2              0      1     0       1      0       0
3              0      1     1       0      0       0
4              1      1     0       1      0       0
5              1      0     1       0      0       0
6              0      1     1       0      0       0
7              1      0     1       0      0       0
8              1      1     1       0      1       0
9              1      1     1       0      0       0
10             0      0     0       0      0       1


In [ ]:
!pip install mlxtend #machine learning extension

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

itemsets = apriori(df1, min_support=0.2, use_colnames=True)
print(itemsets)

    support             itemsets
0       0.6                (Red)
1       0.7              (White)
2       0.6               (Blue)
3       0.2             (Orange)
4       0.2              (Green)
5       0.4         (Red, White)
6       0.4          (Red, Blue)
7       0.2         (Red, Green)
8       0.4        (Blue, White)
9       0.2      (Orange, White)
10      0.2       (Green, White)
11      0.2   (Blue, Red, White)
12      0.2  (Red, Green, White)


In [ ]:
rules = association_rules(itemsets, metric='confidence', min_threshold=0.5)

print(rules.sort_values(by=['lift'], ascending=False))

       antecedents   consequents  antecedent support  consequent support  \
13    (Red, White)       (Green)                 0.4                 0.2   
15         (Green)  (Red, White)                 0.2                 0.4   
4          (Green)         (Red)                 0.2                 0.6   
14  (Green, White)         (Red)                 0.2                 0.6   
7         (Orange)       (White)                 0.2                 0.7   
8          (Green)       (White)                 0.2                 0.7   
12    (Red, Green)       (White)                 0.2                 0.7   
2            (Red)        (Blue)                 0.6                 0.6   
3           (Blue)         (Red)                 0.6                 0.6   
0            (Red)       (White)                 0.6                 0.7   
1          (White)         (Red)                 0.7                 0.6   
5           (Blue)       (White)                 0.6                 0.7   
6          (

In [ ]:
#### collaborative filtering - item-based

In [ ]:
df2 = pd.read_csv('movie_rating.csv')
df2

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
19783,598,5418,4.0,1430616915,"Bourne Identity, The (2002)",Action|Mystery|Thriller
19784,599,5418,3.0,1498525228,"Bourne Identity, The (2002)",Action|Mystery|Thriller
19785,606,5418,4.0,1171736716,"Bourne Identity, The (2002)",Action|Mystery|Thriller
19786,608,5418,4.5,1147794807,"Bourne Identity, The (2002)",Action|Mystery|Thriller


In [ ]:
print('The ratings dataset has', df2['userId'].nunique(), 'unique users')
print('The ratings dataset has', df2['movieId'].nunique(), 'unique movies')
print('The ratings dataset has', df2['rating'].nunique(), 'unique ratings')
print('The unique ratings are', sorted(df2['rating'].unique()))

The ratings dataset has 597 unique users
The ratings dataset has 134 unique movies
The ratings dataset has 10 unique ratings
The unique ratings are [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357251 sha256=6e6a39d5331f0d3f46d3060b593c6c5c2e6eca70008041daf7cf43dbd814d217
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, KNNBasic, accuracy
from surprise.model_selection import train_test_split

#df2를 suprise dataset 형태로 변환
reader = Reader(rating_scale=(0.5, 5.0)) #rating_scale 매개변수를 사용하여 데이터셋의 평점 범위를 지정 ('Reader'는 데이터셋을 읽는 방법을 정의)
data = Dataset.load_from_df(df2[['userId', 'movieId', 'rating']], reader) #Dataset.load_from_df 메소드: pandas dataframe을 surprise 데이터셋 형식으로 변환

#변환된 dataset의 형태를 출력
data_df = pd.DataFrame(df2[['userId', 'movieId', 'rating']])
print(data_df.head())

   userId  movieId  rating
0       1        1     4.0
1       5        1     4.0
2       7        1     4.5
3      15        1     2.5
4      17        1     4.5


In [ ]:
trainset = data.build_full_trainset()

sim_options = {'name': 'pearson', 'user_based': False}
#sim_options 딕셔너리: 유사도를 계산하는 방법과 방향을 설정 (name: person 피어슨상관계수를 유사도 측정 방법으로 사용) (user_based: False 아이템기반협업필터링)
similarity = KNNBasic(sim_options=sim_options)


similarity.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:
def get_top_n_similar_movies(movie_id, n=10): #유사한 영화를 10개 찾도록 설정
    inner_id = similarity.trainset.to_inner_iid(movie_id) #suprise 모델의 학습데이터에서 사용하는 내부 id로 변환
    movie_similarities = similarity.get_neighbors(inner_id, k=n) #내부 영화 id를 기분으로 상위 n개의 유사한 영화의 내부 id를 반환
    movie_similarities = [similarity.trainset.to_raw_iid(inner_id) for inner_id in movie_similarities] #유사한 영화의 내부 id를 원래 영화 id 리스트로 변환
    return movie_similarities

In [ ]:
similar_movies = get_top_n_similar_movies(1, 10)
print("Top 10 movies similar to movie ID 1:")
for movie_id in similar_movies:
    movie_title = df2[df2['movieId'] == movie_id]['title'].iloc[0]
    print(movie_title)

Top 10 movies similar to movie ID 1:
Incredibles, The (2004)
Finding Nemo (2003)
Aladdin (1992)
Monsters, Inc. (2001)
Mrs. Doubtfire (1993)
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
American Pie (1999)
Die Hard: With a Vengeance (1995)
E.T. the Extra-Terrestrial (1982)
Home Alone (1990)


In [ ]:
#아이템기반협업필터링 수동 확인 코드
def calculate_pearson(movie_id_1, movie_id_2):
    #타겟 영화(1), 나머지 영화(2)의 점수 추출
    ratings_1 = df2[df2['movieId'] == movie_id_1][['userId', 'rating']]
    ratings_2 = df2[df2['movieId'] == movie_id_2][['userId', 'rating']]

    #userId로 구분하여 각 영화의 점수를 추출하여 합치고 _1, _2로 구분
    merged_ratings = pd.merge(ratings_1, ratings_2, on='userId', suffixes=('_1', '_2'))

    x = merged_ratings['rating_1']
    y = merged_ratings['rating_2']

    mean_x = np.mean(x)
    mean_y = np.mean(y)

    #피어슨상관계수의 분자, 분모 계산
    numerator = np.sum((x - mean_x) * (y - mean_y))
    denominator = np.sqrt(np.sum((x - mean_x)**2) * np.sum((y - mean_y)**2))

    #피어슨 상관계수 계산
    if denominator == 0:
        return 0
    else:
        pearson_correlation = numerator / denominator
        return pearson_correlation

#타겟영화를 제외한 나머지 영화 리스트 불러오기 (여기서 타겟은 영화1, 토이스토리)
all_movie_ids = df2['movieId'].unique()
target_movie_id = 1

#타겟영화의 이름 저장
target_movie_title = df2[df2['movieId'] == target_movie_id]['title'].iloc[0]

#나머지 영화와의 상관계수 계산하고, 영화id와 상관계수값 저장
correlation_results = []
for movie_id in all_movie_ids:
    if movie_id != target_movie_id:
        pearson_corr = calculate_pearson(target_movie_id, movie_id)
        correlation_results.append((movie_id, pearson_corr))

#입력값 x에 대해 x[1]을 반환하는 것을 의미, correlation_results는 (movie_id, pearson_correlation)이므로 x[1]은 상관계수 값을 의미
correlation_results.sort(key=lambda x: x[1], reverse=True) #reverse=True: 정렬된 값을 내림차순으로 정렬

#결과출력
print(f"Pearson correlation coefficients with movie ID {target_movie_id}({target_movie_title}):")
for movie_id, corr in correlation_results:
    movie_title = df2[df2['movieId'] == movie_id]['title'].iloc[0]
    print(f"Movie ID: {movie_id}, Title: {movie_title}, Pearson Correlation: {corr}")

Pearson correlation coefficients with movie ID 1(Toy Story (1995)):
Movie ID: 8961, Title: Incredibles, The (2004), Pearson Correlation: 0.6433014469670849
Movie ID: 6377, Title: Finding Nemo (2003), Pearson Correlation: 0.6187009323878009
Movie ID: 588, Title: Aladdin (1992), Pearson Correlation: 0.6118923229255582
Movie ID: 4886, Title: Monsters, Inc. (2001), Pearson Correlation: 0.49023122263219737
Movie ID: 500, Title: Mrs. Doubtfire (1993), Pearson Correlation: 0.4462608266903527
Movie ID: 4973, Title: Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001), Pearson Correlation: 0.43823653232037163
Movie ID: 2706, Title: American Pie (1999), Pearson Correlation: 0.420117272100071
Movie ID: 165, Title: Die Hard: With a Vengeance (1995), Pearson Correlation: 0.41093895241876166
Movie ID: 1097, Title: E.T. the Extra-Terrestrial (1982), Pearson Correlation: 0.4092157265940127
Movie ID: 586, Title: Home Alone (1990), Pearson Correlation: 0.4084442748271332
Movie ID: 34, Title: Babe (1995)

In [ ]:
#### collaborative filtering - user-based

In [ ]:
sim_options = {'name': 'pearson', 'user_based': True}  #user_based를 True로 설정하여 사용자 기반 협업 필터링
similarity = KNNBasic(sim_options=sim_options)

similarity.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:
def get_top_n_similar_users(user_id, n=10):  #유사한 사용자를 10명 찾도록 설정
    inner_id = similarity.trainset.to_inner_uid(user_id)  #suprise 모델의 학습데이터에서 사용하는 내부 id로 변환
    user_similarities = similarity.get_neighbors(inner_id, k=n)  #내부 사용자 id를 기준으로 상위 n명의 유사한 사용자의 내부 id를 반환
    user_similarities = [similarity.trainset.to_raw_uid(inner_id) for inner_id in user_similarities]  #유사한 사용자의 내부 id를 원래 사용자 id 리스트로 변환
    return user_similarities

In [ ]:
similar_users = get_top_n_similar_users(1, 10)
print("Top 10 users similar to user ID 1:")
for user_id in similar_users:
    print(f"User ID: {user_id}")

Top 10 users similar to user ID 1:
User ID: 550
User ID: 9
User ID: 108
User ID: 598
User ID: 502
User ID: 401
User ID: 511
User ID: 366
User ID: 595
User ID: 154


In [ ]:
#사용자 기반 협업 필터링 수동 확인 코드
def calculate_pearson(user_id_1, user_id_2):
    # 타겟 사용자(1), 나머지 사용자(2)의 점수 추출
    ratings_1 = df2[df2['userId'] == user_id_1][['movieId', 'rating']]
    ratings_2 = df2[df2['userId'] == user_id_2][['movieId', 'rating']]

    #movieId로 구분하여 각 사용자의 점수를 추출하여 합치고 _1, _2로 구분
    merged_ratings = pd.merge(ratings_1, ratings_2, on='movieId', suffixes=('_1', '_2'))

    x = merged_ratings['rating_1']
    y = merged_ratings['rating_2']

    mean_x = np.mean(x)
    mean_y = np.mean(y)

    #피어슨 상관계수의 분자, 분모 계산
    numerator = np.sum((x - mean_x) * (y - mean_y))
    denominator = np.sqrt(np.sum((x - mean_x)**2) * np.sum((y - mean_y)**2))

    #피어슨 상관계수 계산
    if denominator == 0:
        return 0
    else:
        pearson_correlation = numerator / denominator
        return pearson_correlation

#타겟 사용자를 제외한 나머지 사용자 리스트 불러오기
all_user_ids = df2['userId'].unique()
target_user_id = 1

#나머지 사용자와의 상관계수 계산하고, 사용자 id와 상관계수값 저장
correlation_results = []
for user_id in all_user_ids:
    if user_id != target_user_id:
        pearson_corr = calculate_pearson(target_user_id, user_id)
        correlation_results.append((user_id, pearson_corr))

#입력값 x에 대해 x[1]을 반환하는 것을 의미, correlation_results는 (user_id, pearson_correlation)이므로 x[1]은 상관계수 값을 의미
correlation_results.sort(key=lambda x: x[1], reverse=True)  # reverse=True: 정렬된 값을 내림차순으로 정렬

#결과출력
print(f"Pearson correlation coefficients with user ID {target_user_id}:")
for user_id, corr in correlation_results:
    print(f"User ID: {user_id}, Pearson Correlation: {corr}")

Pearson correlation coefficients with user ID 1:
User ID: 550, Pearson Correlation: 1.0
User ID: 108, Pearson Correlation: 1.0
User ID: 598, Pearson Correlation: 1.0
User ID: 502, Pearson Correlation: 1.0
User ID: 9, Pearson Correlation: 0.9999999999999999
User ID: 401, Pearson Correlation: 0.9428090415820635
User ID: 511, Pearson Correlation: 0.9258200997725514
User ID: 366, Pearson Correlation: 0.8728715609439694
User ID: 154, Pearson Correlation: 0.8660254037844387
User ID: 595, Pearson Correlation: 0.8660254037844385
User ID: 90, Pearson Correlation: 0.8385254915624212
User ID: 44, Pearson Correlation: 0.8107121179967625
User ID: 430, Pearson Correlation: 0.8017837257372734
User ID: 114, Pearson Correlation: 0.790569415042095
User ID: 605, Pearson Correlation: 0.7800202620757808
User ID: 392, Pearson Correlation: 0.7745966692414834
User ID: 487, Pearson Correlation: 0.7745966692414833
User ID: 530, Pearson Correlation: 0.7559289460184544
User ID: 49, Pearson Correlation: 0.75000000

In [ ]:
user_id_1 = 1
user_id_2 = 550
ratings_1 = df2[df2['userId'] == user_id_1][['movieId', 'rating']]
ratings_2 = df2[df2['userId'] == user_id_2][['movieId', 'rating']]
merged_ratings = pd.merge(ratings_1, ratings_2, on='movieId', suffixes=('_1', '_2'))
merged_ratings = pd.merge(merged_ratings, df2[['movieId', 'title']].drop_duplicates(), on='movieId')
comparison_table = merged_ratings[['title', 'rating_1', 'rating_2']]
comparison_table.columns = ['Movie Title', 'User 1 Rating', 'User 550 Rating']
print(comparison_table)

           Movie Title  User 1 Rating  User 550 Rating
0     Toy Story (1995)            4.0              4.0
1  Forrest Gump (1994)            4.0              4.0
2   Matrix, The (1999)            5.0              4.5


In [ ]:
user_id_1 = 1
user_id_2 = 108
ratings_1 = df2[df2['userId'] == user_id_1][['movieId', 'rating']]
ratings_2 = df2[df2['userId'] == user_id_2][['movieId', 'rating']]
merged_ratings = pd.merge(ratings_1, ratings_2, on='movieId', suffixes=('_1', '_2'))
merged_ratings = pd.merge(merged_ratings, df2[['movieId', 'title']].drop_duplicates(), on='movieId')
comparison_table = merged_ratings[['title', 'rating_1', 'rating_2']]
comparison_table.columns = ['Movie Title', 'User 1 Rating', 'User 108 Rating']
print(comparison_table)

                         Movie Title  User 1 Rating  User 108 Rating
0               Jurassic Park (1993)            4.0              4.0
1  E.T. the Extra-Terrestrial (1982)            5.0              5.0
2          Back to the Future (1985)            5.0              5.0
3                 Matrix, The (1999)            5.0              5.0
